# Project: Webscraping Indeed 

#### Full code is at the end


# Load Libraries

In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Exploring the Html code 
1. Try different methods in beautiful soup to get a feel for how to scrape the data

 When you use requests you only get the HTML content of the page which doesn't have what you need. You should use for example selenium (or requests-html) if you want to parse page with elements rendered by web browser.
 
 https://stackoverflow.com/questions/55351871/beautifulsoup-attributeerror-nonetype-object-has-no-attribute-text

In [20]:
# Exploring the html
url = f'https://www.indeed.com/jobs?l=Bronx,%20NY&radius=10&explvl=entry_level&fromage=14&start=0&vjk=2dd1503812377c92'
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36 Edg/99.0.1150.46'}
Indeed = requests.get(url, headers)
soup = BeautifulSoup(Indeed.content, 'html.parser')
Job_table = soup.find(class_="slider_item")
table_body = Job_table.find('tbody')
table_item =table_body.find('td')
Company = table_item.find(class_="companyName")
Job_title = table_item.find(class_ ="jobTitle")
Locaton = table_item.find(class_="companyLocation")

print(Company.get_text())
print(Job_title.get_text())
print(Locaton.get_text()) # the +4 represents other locations


Indeed Gigs
newFind jobs with Flexible Hours!
Bronx, NY+4 locations


# Make the code into a Function  for easier use.

In [4]:
# time to check response
# from what I've seen, it should be fine to webscrape indeedd

def Choose(page):
    ''' 1st page = 0, pages afterwards goes up by 10.
        some sites need more information before they let allow you to webscrape
        use  user agent
    '''
    url = f'https://www.indeed.com/jobs?l=Bronx,%20NY&radius=10&explvl=entry_level&fromage=14&start={page}&vjk=2dd1503812377c92'
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36 Edg/99.0.1150.46'}
    Indeed = requests.get(url, headers)
    soup = BeautifulSoup(Indeed.content, 'html.parser')
    return soup



In [21]:
# trying CSS selectors
item = Job_table.select("tbody tr td")
# returns everything in one list
infos = [pt.get_text() for pt in item]
infos

['newFind jobs with Flexible Hours!Indeed GigsBronx, NY+4 locationsPart-time +2',
 '\nWhether it’s hiring for online work from home, part time, entry level or no experience jobs, you get to set your own schedule!\nPostedToday']

In [17]:
def transform(soup):
    divs = soup.find_all('div',class_="slider_item")
    for item in divs:
        title = item.find(attrs={'class':'jobTitle'}).text.strip()
        company = item.find(attrs={'class':'companyName'}).text.strip()
        Location = item.find(attrs={'class':'companyLocation'}).text.strip()
        try:
            salary = item.find(attrs={'class':'metadata estimated-salary-container'}).text.strip()
        except:
            salary ='NA'
        Description = item.find(attrs={'class':'job-snippet'}).text.strip().replace('\n','')
        try:
            employment_a = item.find(attrs={'class':'attribute_snippet'}).text.strip()
        except:
            employment_a ='NA'
        Posted = item.find(attrs={'class':'date'}).text.strip()
        
        # create a Dict to dataframe use 
        Jobs = {'Title':title,
                'Company' : company,
                'Location' : Location,
                'Salary' : salary,
                'Description' : Description,
                'Job_Type' :  employment_a,
                'Date_posted' : Posted}
        joblist.append(Jobs)
    return
   
joblist = []
# could later change the 40 to a input function
#as this code will get you 3 pages worth.

for i in range(0,40, 10):
    print(f'Getting pages, {i}')
    c = Choose(i)
    transform(c)

    
Indeed_df = pd.DataFrame(joblist)    
    
Indeed_df.to_csv('Indeedjobs.csv')

Getting pages, 0
Getting pages, 10
Getting pages, 20
Getting pages, 30


# Method of Choice

Finding a scraping method that straight forward , 
using attrs parameter seems to works pretty well.
it allows us to find  or desired data by searching by attributes
 using a dictionary, the key is  the tag and value in a this case is string
 class = 'jobtitle'. This  method is present in both  find and find all

In [22]:
# test
jobs_a = soup.find_all(attrs={'class':'jobTitle'})
Scrapped_titles = []
for job  in jobs_a:
    Titles = job.get_text()
    Scrapped_titles.append(Titles)

Scrapped_titles

['newFind jobs with Flexible Hours!',
 'Door Attendant',
 'Overnight- Houseman, Bellmen',
 'Housemen, Bellmen',
 'newDental Receptionist FT No Evenings No Weekends',
 'newMedical Receptionist/Assistant',
 'newNow Hiring! Front Desk Associates - West Fordham',
 'Overnight Door Attendant',
 'newFull-Time Store Associate',
 'Cashier',
 'newCOVID-19 Screener',
 'newProgram Associate',
 'newPatient Assistant',
 'newTeam Lead',
 'newCustomer Experience Associate']

# Further Exploring Each Html Tag of interest

In [38]:
esalary_a = soup.find_all(attrs={'class':'metadata estimated-salary-container'})
Scrapped_Salary = []
for est in esalary_a :
    C_salary =  est.get_text()
    Scrapped_Salary.append(C_salary)
# need to put in a deafult of  NA, as its not usable right now    
Scrapped_Salary




['Estimated $24.4K – $31K a year',
 'Estimated $24.8K – $31.4K a year',
 'Estimated $24.8K – $31.4K a year']

### Missing Values

At first I was confused at first, whyI had so many values returning when trying to get the attributes for each job (like partime etc.) ?  The answer is simple , not every job post its salary, and 
similarilty some jobs might have mutiple labels(attributes), others might not have any
so either exluded these or create  a code that gives a default value,while also stop
value error, due to NoneType


In [26]:
#
# job Skill dsecript
employment_a = soup.find_all(attrs={'class':'attribute_snippet'})
Scrapped_employment = []
for contract in employment_a :
    jobe= contract.get_text()
    Scrapped_employment.append(jobe)
Scrapped_employment

['Part-time +2',
 '$35 an hour',
 'Full-time',
 '8 hour shift',
 '$26 an hour',
 'Full-time',
 '8 hour shift +1',
 '$25 an hour',
 'Full-time',
 '8 hour shift',
 '$18 - $26 an hour',
 'Full-time',
 '$19 - $22 an hour',
 'Full-time',
 'Part-time',
 '$40 an hour',
 'Full-time',
 '8 hour shift +1',
 '$18 an hour',
 'Full-time',
 'Part-time',
 'Day shift',
 'Full-time',
 'Full-time',
 'Evening shift',
 'From $31.25 an hour',
 'Full-time',
 '8 hour shift +2',
 'Contract']

# Full code

In [31]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


def Choose(page):
    ''' 1st page = 0, pages afterwards goes up by 10.
        some sites need more information before they let allow you to webscrape
        use  user agent
    '''
    url = f'https://www.indeed.com/jobs?l=Bronx,%20NY&radius=10&explvl=entry_level&fromage=14&start={page}&vjk=2dd1503812377c92'
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36 Edg/99.0.1150.46'}
    Indeed = requests.get(url, headers)
    soup = BeautifulSoup(Indeed.content, 'html.parser')
    return soup


def transform(soup):
    divs = soup.find_all('div',class_="slider_item")
    for item in divs:
        title = item.find(attrs={'class':'jobTitle'}).text.strip()
        company = item.find(attrs={'class':'companyName'}).text.strip()
        Location = item.find(attrs={'class':'companyLocation'}).text.strip()
        try:
            salary = item.find(attrs={'class':'metadata estimated-salary-container'}).text.strip()
        except:
            salary ='NA'
        Description = item.find(attrs={'class':'job-snippet'}).text.strip().replace('\n','')
        try:
            employment_a = item.find(attrs={'class':'attribute_snippet'}).text.strip()
        except:
            employment_a ='NA'
        Posted = item.find(attrs={'class':'date'}).text.strip()
        
        # create a Dict to dataframe use 
        Jobs = {'Title':title,
                'Company' : company,
                'Location' : Location,
                'Salary' : salary,
                'Description' : Description,
                'Job_Type' :  employment_a,
                'Date_posted' : Posted}
        joblist.append(Jobs)
    return
   
joblist = []
# could later change the 40 to a input function
#as this code will get you 3 pages worth.

for i in range(0,40, 10):
    print(f'Getting pages, {i}')
    c = Choose(i)
    transform(c)

    
Indeed_df = pd.DataFrame(joblist)    
    
Indeed_df.to_csv('Indeedjobs.csv',index=False)


Getting pages, 0
Getting pages, 10
Getting pages, 20
Getting pages, 30
